# Text Classification Using Embeddings


## 1. Get the dataset

In [1]:
import pandas as pd
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

# Get the SST2 training and test sets
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [2]:
# Let's glance at the dataset
df.head()

,0,1
0,"a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films",1
1,apparently reassembled from the cutting room floor of any given daytime soap,0
2,"they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science fiction elements of bug eyed monsters and futuristic women in skimpy clothes",0
3,"this is a visually stunning rumination on love , memory , history and the war between art and commerce",1
4,jonathan parker 's bartleby should have been the be all end all of the modern office anomie films,1


We'll only use a subset of the training and testing datasets in this example. We'll only use 100 examples since this is a toy example. You'll want to increase the number to get better performance and evaluation. 

In [3]:
num_examples = 500
df_sample = df.sample(num_examples)

# Split into training and testing sets
sentences_train, sentences_test, labels_train, labels_test = train_test_split(
            list(df_sample[0]), list(df_sample[1]), test_size=0.25, random_state=0)

## 2. Get the embeddings of the reviews


In [4]:

api_key = "Uuk4dd4AYisFJfrG4WA4jVlLfmYq2oVtsZlbEnyM"

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [6]:
# Embed the training set
embeddings_train = co.embed(texts=sentences_train,
                             model="large",
                             truncate="LEFT").embeddings
# Embed the testing set
embeddings_test = co.embed(texts=sentences_test,
                             model="large",
                             truncate="LEFT").embeddings

In [7]:
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Review text: the threat implied in the title pok mon 4ever is terrifying like locusts in a horde these things will keep coming
Embedding vector: [2.7435012, -0.03950072, 1.2659788, 0.51788735, 1.7879175, 0.95217633, 0.8021472, -0.64531827, 0.008974285, 0.43134677]


## 3. Train a classifier using the training set


In [8]:
# import SVM classifier code
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# Initialize a support vector machine, with class_weight='balanced' because 
# our training set has roughly an equal amount of positive and negative 
# sentiment sentences
svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 

# fit the support vector machine
svm_classifier.fit(embeddings_train, labels_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced'))])

## 4. Evaluate the performance of the classifier on the testing set

In [10]:
# get the score from the test set, and print it out to screen!
score = svm_classifier.score(embeddings_test, labels_test)
print(f"Validation accuracy on Large is {100*score}%")

Validation accuracy on Large is 86.4%
